In [1]:
import sys
sys.path.append('..')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cupy as _cu
from tqdm.auto import tqdm
import numpy as np
import math

In [4]:
import scaredcu

/home/toluntosun/miniconda3/envs/sca/lib/python3.12/site-packages/cupy/_creation/from_data.py:53: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 3562152210 to int32 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  return _core.array(obj, dtype, copy, order, subok, ndmin, blocking)
/home/toluntosun/miniconda3/envs/sca/lib/python3.12/site-packages/cupy/_creation/from_data.py:53: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -3562152210 to int32 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  return _core.array(obj, dtype, copy, order, subok, ndmin, blocking)
/home/toluntosun/miniconda3/envs/sca/lib/python3.12/site-packages/cupy/_creation/from_dat

In [5]:
reduction = scaredcu.lbc.modop.Reduction_0Q(8380417, 'int32')

model = scaredcu.models.AbsoluteValue()


N = 900
sigma = 0
conv = 50

# scared_cu.container.set_batch_size(5)

n = 8 #don't touch
n_step = 1

neg_trick = False

prng_off = False

incomplete = 0

cu_step = 1 << 21

In [6]:
def hocpa_wrapper(ths, reduction, model, step=1000, conv_step=1000, neg_trick=False, subkeys=None, early_stop=True, pp=True):


    selection_function = scaredcu.lbc.selection_functions.BaseMulLargeQ(reduction=reduction, words=subkeys, cu_step=cu_step)

    selection_function.reset()

    for _ in tqdm(range(selection_function.num_steps)):
        assert selection_function.done() == False


        cpa = scaredcu.CPAAttackAlternative(selection_function=selection_function,
                                                model=model,
                                                discriminant=scaredcu.maxabs,
                                                convergence_step=conv_step)


        mean = ths.samples.mean(axis=0)
        _n = ths.samples.shape[1] // 2
        pp_ = scaredcu.preprocesses.high_order.CenteredProduct(mean=mean, frame_1=range(0,_n), frame_2=range(_n,2*_n), mode='same')

        for i in range(0, len(ths), step):
            if pp:
                container = scaredcu.Container(ths[i:i + step], preprocesses=[pp_])
            else:
                container = scaredcu.Container(ths[i:i + step])
            cpa.run(container)
            # if early_stop:
            #     succs = util.succ_ratio(ths.s[0, subkeys], reduction.q, conv, cpa)
            #     if succs[-1] == 1.0:
            #         break

        selection_function.save_scores(cpa.convergence_traces)
        selection_function.next()
    return selection_function.scores

In [7]:
ths = scaredcu.lbc.simulated_traces.collect_traces_basemult(N=N, n=n, sigma=sigma, reduction=reduction, incomplete=incomplete,
                              const_seed=True, two_step=False, seed=0, prng_off=prng_off, leak_reduction=True)

print(ths)

  0%|          | 0/900 [00:00<?, ?it/s]

Trace Header Set:
Name.............: RAM Format THS
Reader...........: RAM reader with 900 traces. Samples shape: (900, 16) and metadatas: ['c', 's', 's0', 's1']
c................: int32
s................: int32
s0...............: int32
s1...............: int32



In [8]:

succs = None
for n_i in tqdm(range(0, n, n_step)):
    subkeys = _cu.arange(n_i, n_i + n_step)
    assert ((ths.s % reduction.q) == ((ths.s0 + ths.s1) % reduction.q)).all()
    cpa_scores = hocpa_wrapper(ths[:N], reduction, model, step=conv, conv_step=conv, neg_trick=neg_trick, subkeys=subkeys, early_stop=False, pp=not prng_off)
    succs_i_ = scaredcu._utils.succ_ratio(ths.s[0, subkeys], reduction.q, conv, cpa_scores, incomplete=incomplete!=0, np_flag=True)
    succs_i = _cu.zeros(N//conv)
    for i in range(len(succs_i_)):
        succs_i[i] = succs_i_[i]
    for i in range(len(succs_i_), len(succs_i)):
        succs_i[i] = succs_i_[-1]
    if succs is None:
        succs = _cu.array(succs_i)
    else:
        succs += _cu.array(succs_i)
    
    for x,y in zip(range(conv,N+1,conv),succs):
        print(f'({math.log(x)/math.log(10)},{y})',end='')
    # if n_i != (n - n_step):
    #     del kw
    print()
    for i in range(n_i, n_i + n_step):
        print(f'{ths.s[0, i]}-{cpa_scores[:,i-n_i].argsort()[-1]}', end='\t')
    print()



succs /= (n//n_step)
# assert (mmia.data == (-1)).sum() == 0


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
for x,y in zip(range(conv,N+1,conv),succs):
    print(f'({math.log(x)/math.log(10)},{y})',end='')

(2.397940008672037,0.01)(2.6989700043360183,0.01)(2.8750612633916997,0.01)(2.9999999999999996,0.01)(3.096910013008056,0.01)(3.176091259055681,0.01)(3.243038048686294,0.01)(3.301029995663981,0.01)